In [ ]:
class To_Uniform(nn.Module):
    def __init__(self,
                 encoder_layers=[3, 128, 64, 32, 16, 8, 2],
                 decoder_layers=[2, 8, 16, 32, 64, 128, 3],
                 encoder_act=nn.ELU,
                 decoder_act=nn.ELU,
                 final_encoder_act=nn.Sigmoid,
                 final_decoder_act=nn.Sigmoid,
                 use_batchnorm=True):
        super().__init__()

        self.encoder = self.build_mlp(encoder_layers, encoder_act, final_encoder_act, use_batchnorm)
        self.decoder = self.build_mlp(decoder_layers, decoder_act, final_decoder_act, use_batchnorm=False)

    def build_mlp(self, layer_sizes, activation_fn, final_activation, use_batchnorm=False):
        layers = []
        for i in range(len(layer_sizes) - 2):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            layers.append(activation_fn())
            if use_batchnorm:
                layers.append(nn.BatchNorm1d(layer_sizes[i+1]))
        # Son katman
        layers.append(nn.Linear(layer_sizes[-2], layer_sizes[-1]))
        if final_activation is not None:
            layers.append(final_activation())
        return nn.Sequential(*layers)

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z) + torch.randn(x.shape) * 0.01
        return z, x_hat

    def decode(self, z):
        x_decoded = self.decoder(z)
        x_hat = x_decoded + torch.randn(x_decoded.shape) * 0.01
        return x_hat

    def criterion(self, z_pred, z_true, x_pred, x_true):
        return - 0.4 * nn.MSELoss()(z_pred, z_true) + 0.6 * nn.MSELoss()(x_pred, x_true)